## QAOA Parameter Transfer: Graph A (Baseline)

### 1. Setup
- Imports
- Graph drawing function
- Helper utils

### 2. Graph A: MaxCut Formulation
- Create graph
- Show adjacency + plot

### 3. Convert to QUBO + Ising
- Use Qiskit Optimization

### 4. QAOA Ansatz
- Create and show circuit (with measurement)
- Create circuit (no measurement) for Estimator

### 5. QAOA Optimization
- Define cost function
- Run COBYLA
- Save trajectory

### 6. Results
- Print final energy + parameters
- Plot cost vs iteration
- Save outputs (JSON/PNG)



### Setup

In [2]:
%pip install qiskit==1.1.0 qiskit_optimization==0.6.1 matplotlib networkx pylatexenc --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
from pathlib import Path

from qiskit_optimization.applications import Maxcut
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.primitives import Estimator
from qiskit.circuit.library import QAOAAnsatz
from scipy.optimize import minimize

%matplotlib inline

### Graph A - Build and Visualize

In [ ]:
# Graph A: Complete 4-node
G = nx.complete_graph(4)
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(4, 4))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=600, edge_color='gray')
plt.title("Graph A: Complete 4-Node")
plt.show()
